# Lab 01: Gurobi Basics

Made & Presented by Bo Tang

In this lab, you will explore constrained optimization with Gurobi and Python. You will gain hands-on experience with gurobipy and solve optimization problems

## Install

First, we need install Gurobi, apply the academic license [here](https://www.gurobi.com/).

Then, we can install GurobiPy via Pip.

In [ ]:
! pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB

## Linear Programming Example

We will start by solving a simple linear programming problem:

**Maximize**  
$$z = 5x + 3y$$

**subject to**  
- $$2x + y \leq 6$$  
- $$x + 2y \leq 6$$
- $$x, y \geq 0$$

In [ ]:
# Create a new model
model = gp.Model("lp_example")

# Add variables
x = model.addVar(name="x")  # x >= 0 is default lower bound
y = model.addVar(name="y")  # y >= 0 is default lower bound

# Set objective function
model.setObjective(5*x + 3*y, GRB.MAXIMIZE)

# Add constraints
model.addConstr(2*x + y <= 6, "c0")
model.addConstr(x + 2*y <= 6, "c1")

# Solve the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print(f"x: {x.X}")
    print(f"y: {y.X}")
    print(f"Optimal objective value: {model.ObjVal}")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xd7f6b1a8
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 6e+00]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   3.000000e+30   8.000000e+00      0s
       2    1.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.600000000e+01
x: 2.0
y: 2.0
Optimal objective value: 16.0


### Mixed-Integer Programming (MIP)

Now, we introduce how to solve a mixed-integer programming (MIP) problem where variables can only take integer values.

For example, let's solve:

**Maximize**  
$$z = 4x_1 + 6x_2 $$

**subject to**  
- $$x_1 + x_2 \leq 5$$
- $$x_1, x_2 \in \{0, 1, 2, 3, 4, 5\}$$

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model
mip_model = gp.Model("mip_example")

# Add integer variables
x1 = mip_model.addVar(vtype=GRB.INTEGER, name="x1")
x2 = mip_model.addVar(vtype=GRB.INTEGER, name="x2")

# Set objective function
mip_model.setObjective(4*x1 + 6*x2, GRB.MAXIMIZE)

# Add constraints
mip_model.addConstr(x1 + x2 <= 5, "constraint1")

# Optimize the model
mip_model.optimize()

# Print the results
if mip_model.status == GRB.OPTIMAL:
    print(f"x1: {x1.X}")
    print(f"x2: {x2.X}")
    print(f"Optimal objective value: {mip_model.ObjVal}")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xe122815f
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+00]
Found heuristic solution: objective 20.0000000
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 30 20 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+01, best bound 3.000000000000e+01, gap 0.0000%
x1: -0.0
x2: 5.0
Optimal objective value: 30.0


## Sensitivity Analysis

Once you've solved a linear programming problem, you can access additional information, such as shadow prices or dual values.

Here’s an example of how to retrieve and print this information:

In [ ]:
for constr in model.getConstrs():
    print(f"{constr.ConstrName}: Shadow Price = {constr.Pi}")

c0: Shadow Price = 2.3333333333333335
c1: Shadow Price = 0.3333333333333333


## Nonlinear Programming (NLP)

Gurobi can handle certain types of nonlinear programming models. However, nonlinear constraints need to be reformulated for compatibility with Gurobi. Let’s consider a simple nonlinear problem:

In [ ]:
nlp_model = gp.Model("nonlinear_example")

# Add variables
x = nlp_model.addVar(name="x")
y = nlp_model.addVar(name="y")

# Add nonlinear objective (e.g., maximizing x^2 + y^2)
nlp_model.setObjective(x**2 + y**2, GRB.MAXIMIZE)

# Add a constraint
nlp_model.addConstr(x + y <= 10, "c0")

# Optimize
nlp_model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xd2d7cf7f
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 9 nonzeros
Presolved model has 2 bilinear constraint(s)
Variable types: 5 continuous, 0 integer (0 binary)
Found heuristic solution: objective 50.0000000

Root relaxation: objective 1.000000e+02, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bound

## Conclusion

This tutorial provided a brief introduction to solving optimization problems using GurobiPy, including linear programming, mixed-integer programming, and sensitivity analysis. For more advanced uses, such as nonlinear optimization, refer to the Gurobi documentation.